In [1]:
import string
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.image import ImageWriter
from pdfminer.layout import LAParams

import io
import os

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

import json
from pathlib import *

import time

from random import randint

from nltk.corpus import wordnet

import operator

from nltk.chunk import tree2conlltags
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

import re

import numpy as np

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Pooja Ajit\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to C:\Users\Pooja
[nltk_data]     Ajit\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to C:\Users\Pooja
[nltk_data]     Ajit\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
def pdfRender():
    global documentSet
    global mydoc
    mydoc ={}
    pdf_files =[]
    allLines =[]
    FILE_PATH = Path(r'SLR1_EVALUATION_PAPERS/FinalSetforQA')
    #FILE_PATH = Path('E:/MasterThesis/FinalPapers')
    pdf_files = list(FILE_PATH.glob('*.pdf'))
    #An Array which stores the full text of each document
    documentSet = pdfparser(pdf_files)
    mydoc = dict(zip(pdf_files,documentSet))
    #print(len(documentSet))
    return documentSet

In [3]:
def processData(rawContents):    
    cleaned = tokenizeContent(rawContents)    
    cleaned1 = removeStopWordsFromTokenized(cleaned)    
    cleaned2 = performPorterStemmingOnContents(cleaned1)    
    cleaned3 = removePunctuationFromTokenized(cleaned2)    
    cleaned4 = convertItemsToLower(cleaned3)    
    return cleaned4    
        
def tokenizeContent(contentsRaw):    
    tokenized = nltk.tokenize.sent_tokenize(contentsRaw)    
    return tokenized    
    
def removeStopWordsFromTokenized(contentsTokenized):    
    stop_word_set = set(nltk.corpus.stopwords.words("english"))    
    filteredContents = [word for word in contentsTokenized if word not in stop_word_set]    
    return filteredContents    
    
def performPorterStemmingOnContents(contentsTokenized):    
    porterStemmer = nltk.stem.PorterStemmer()    
    filteredContents = [porterStemmer.stem(word) for word in contentsTokenized]    
    return filteredContents    
    
def removePunctuationFromTokenized(contentsTokenized):    
    excludePuncuation = set(string.punctuation)    
    
    # manually add additional punctuation to remove    
    doubleSingleQuote = '\'\''    
    doubleDash = '--'    
    doubleTick = '``'    
    
    excludePuncuation.add(doubleSingleQuote)    
    excludePuncuation.add(doubleDash)    
    excludePuncuation.add(doubleTick)    
    
    filteredContents = [word for word in contentsTokenized if word not in excludePuncuation]    
    return filteredContents    
    
def convertItemsToLower(contentsRaw):    
    filteredContents = [term.lower() for term in contentsRaw]    
    return filteredContents


In [4]:
global fileName
def pdfparser(pdffileS):
    global finalDocumentSet
    finalDocumentSet = []
    global pdfEx
    pdfEx = []
    global files
    global fullText
    files = []
    fileName = []
    for pdffile in pdffileS:
        #full= fullText
        # Create a example words list(Please add all the related keywords needed)
        words_list = ["Introduction", "INTRODUCTION", "Background", "BACKGROUND", "Conclusion", "Conclusions",
                      "CONCLUSION", "Acknowledgements"]
        #print(words_list)
        pdfName = os.path.basename(pdffile)
        files.append(pdfName)
        fileName.append(pdfName)
        with open(pdffile, mode='rb') as f:
            fullText = np.array([])
            print(pdfName)
            #files.append(pdfName)
            #documents = fullText
            #words_list = []
            #print(words_list)
            #fp = open(data, 'rb')
            rsrcmgr = PDFResourceManager()
            retstr = io.StringIO()
            codec = 'utf-8'
            laparams = LAParams()
            data =[]
            details_page = []
            abstract = []
            device = TextConverter(rsrcmgr, retstr, laparams=laparams)
            # Create a PDF interpreter object.
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            # Process each page contained in the document.
            count = 0
            for page in PDFPage.get_pages(f):
                interpreter.process_page(page)
                data = retstr.getvalue()
                details_page.append(data)

            #print("There are", len(words_list), "in the words list")
            stri = " "
            details = stri.join(details_page)
            words = details.split()
            place = []
            dummy_check = []
            removed_words = []

            print(words_list)
            for c, a in enumerate(words):
                for b in words_list:
                    if b == a and b not in dummy_check:
                        print(b, a)
                        place.append(details.find("{}".format(b)))
                        dummy_check.append(b)
                    #  place.append(words.index(a))
                    elif b not in words:
                        print(b)
                        removed_words.append(b)
                        words_list.remove(b)
                        print("The word", b, "was not found in the pdf file")

            #print(list(zip(words_list, place)))
            final_array = list(zip(words_list, place))
            #final_array.sort()
            final_array.sort(key=operator.itemgetter(1))
            # print("Sorting the final array")
            #print(final_array)

            # print("Extracting the relevant texts from pdf")
            # print(" ")
            print(final_array)
            if len(final_array) > 1:
                listint = final_array[0]
                list2int = final_array[1]
                counter = 0

                for each in (final_array):
                    if counter < len(final_array) - 2:
                        new = (details.split(listint[0])[1].split(list2int[0])[0])
                        #new = sent_tokenize(new)
                        #print(listint[0], ":", [' '.join(new)])
                        #print(" ")
                        #print(new)
                        #documents.append(new)
                        fullText = np.append(fullText, new)
                        counter = counter + 1
                        listint = final_array[0 + counter]
                        list2int = final_array[1 + counter]

                    elif counter < len(final_array) - 1:
                        new = (details.split(final_array[counter][0])[1].split(final_array[counter + 1][0])[0])
                        #new = sent_tokenize(new)
                        #documents.append(new)
                        fullText = np.append(fullText, new)
                        #print(final_array[counter][0], ":", [' '.join(new)])
                        #print(" ")
                        counter = counter + 1

                    else:
                        new = (details.split(final_array[counter][0])[1])
                        #new = sent_tokenize(new)
                        #documents.append(new)
                        fullText = np.append(fullText, new)
                        #print(final_array[counter][0], ":", [' '.join(new)])
                        #print(" ")
            else:
                new = (details.split(final_array[0][0])[1])
                # new = sent_tokenize(new)
                #documents.append(new)
                fullText = np.append(fullText, new)
                # print(final_array[counter][0], ":", [' '.join(new)])
                # print(" ")
                
        #finalDocumentSet = {pdfName : fullText}
        
        data=finalDocumentSet.append(fullText)
        myName=pdfEx.append(pdfName)
        #print("Testing==",finalDocumentSet)
        #data = finalDocumentSet.get(pdfName)
        #finalDocumentSet = finalDocumentSet
        data = str(data)
        
        data = processData(data)
        #data = data.replace(r'\\n', "")
        data = [i.replace('\\n', "") for i in data]
        data = [i.replace('\\x0', "") for i in data]
        words_list = words_list + removed_words
        print("Updated words list:")
        print(words_list)

    #print(len(finalDocumentSet))
    
    #mydoc = dict(zip(myName,data))
    return data


In [5]:
pdfRender()

An_autonomous_agent_development_environment_for_engineering_applications.pdf
['Introduction', 'INTRODUCTION', 'Background', 'BACKGROUND', 'Conclusion', 'Conclusions', 'CONCLUSION', 'Acknowledgements']
INTRODUCTION
The word INTRODUCTION was not found in the pdf file
BACKGROUND
The word BACKGROUND was not found in the pdf file
CONCLUSION
The word CONCLUSION was not found in the pdf file
Background
The word Background was not found in the pdf file
Conclusion
The word Conclusion was not found in the pdf file
Introduction Introduction
Conclusions Conclusions
Acknowledgements Acknowledgements
[('Introduction', 1465), ('Conclusions', 186484), ('Acknowledgements', 302786)]
Updated words list:
['Introduction', 'Conclusions', 'Acknowledgements', 'INTRODUCTION', 'BACKGROUND', 'CONCLUSION', 'Background', 'Conclusion']
An_experimental_card_game_for_teaching_software_engineering_processes.pdf
['Introduction', 'INTRODUCTION', 'Background', 'BACKGROUND', 'Conclusion', 'Conclusions', 'CONCLUSION', 'Ack

['Introduction', 'INTRODUCTION', 'Background', 'BACKGROUND', 'Conclusion', 'Conclusions', 'CONCLUSION', 'Acknowledgements']
INTRODUCTION
The word INTRODUCTION was not found in the pdf file
BACKGROUND
The word BACKGROUND was not found in the pdf file
Conclusions
The word Conclusions was not found in the pdf file
Acknowledgements
The word Acknowledgements was not found in the pdf file
Background
The word Background was not found in the pdf file
CONCLUSION
The word CONCLUSION was not found in the pdf file
Conclusion
The word Conclusion was not found in the pdf file
Introduction Introduction
[('Introduction', 2427)]
Updated words list:
['Introduction', 'INTRODUCTION', 'BACKGROUND', 'Conclusions', 'Acknowledgements', 'Background', 'CONCLUSION', 'Conclusion']
Experience_with_software_process_simulation_and_modeling.pdf
['Introduction', 'INTRODUCTION', 'Background', 'BACKGROUND', 'Conclusion', 'Conclusions', 'CONCLUSION', 'Acknowledgements']
INTRODUCTION
The word INTRODUCTION was not found in

['Introduction', 'INTRODUCTION', 'Background', 'BACKGROUND', 'Conclusion', 'Conclusions', 'CONCLUSION', 'Acknowledgements']
INTRODUCTION
The word INTRODUCTION was not found in the pdf file
BACKGROUND
The word BACKGROUND was not found in the pdf file
Conclusions
The word Conclusions was not found in the pdf file
Acknowledgements
The word Acknowledgements was not found in the pdf file
CONCLUSION
The word CONCLUSION was not found in the pdf file
Introduction Introduction
Background Background
Conclusion Conclusion
[('Introduction', 1911), ('Background', 9500), ('Conclusion', 363917)]
Updated words list:
['Introduction', 'Background', 'Conclusion', 'INTRODUCTION', 'BACKGROUND', 'Conclusions', 'Acknowledgements', 'CONCLUSION']
Software_process_white_box_modelling_for_FEAST1.pdf
['Introduction', 'INTRODUCTION', 'Background', 'BACKGROUND', 'Conclusion', 'Conclusions', 'CONCLUSION', 'Acknowledgements']
Introduction
The word Introduction was not found in the pdf file
BACKGROUND
The word BACKGRO

['none']

In [6]:
len(finalDocumentSet)

28

In [115]:
finalDocumentSet[0]

array(['\n\nComplex engineering design projects generally require\nthe cooperation of multidisciplinary design teams and the\nreadily accessibility of various engineering tools, such as\nCAD, FEA, dynamic and kinematic analysis, and simulation\npackages. However, the signiﬁcant investments and high-\ndemanding expertise of these engineering tools hinder the\nsmall and medium enterprises from acquiring their design\nand engineering competencies. Apart from economical\nconsiderations, other forbidding factors in product design\nand engineering process include:\n\n– The time consuming nature of product design and\nengineering process, which may contain design\nloops,\niterations, choice between alternatives,\ndesign optimization, etc.;\n\n– Heterogeneous and geographically distributed\nengineering software tools, personnel, knowledge,\nand engineering data;\n\n* Corresponding author. Tel.: C1 519 430 7149; fax: C1 519 430 7064.\nE-mail addresses: qi.hao@nrc.gc.ca (Q. Hao), weiming.shen@nr

In [134]:
len(files)


28

In [135]:
files[0]

'An_autonomous_agent_development_environment_for_engineering_applications.pdf'

In [136]:
#create a dictionary to save the pdf name and the objectives for QA questions
final_dataset_QA = {'fileName': [], 'content': [] }
for a,b in zip(files, finalDocumentSet):    
    final_dataset_QA['fileName'].append(a)
    final_dataset_QA['content'].append(b)

In [137]:
#export the results to .xslx
from pandas import DataFrame

df = DataFrame(final_dataset_QA, columns= ['fileName', 'content'])
export_excel = df.to_excel (r'E:\MasterThesis\EVALUATION\SLR_evaluation\New folder\SLR1\QA_evaluation\SLR1_Dataset.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path



In [11]:
#extract introduction, methods, conclusion portions of the paper
extractedContent = []
global keyword
def extractIntroduction():
    count = 0
    for i in range(len(finalDocumentSet)):
        doc = str(finalDocumentSet[i])
        intro=re.search("|".join([r'Background(.*?)Methods', r'2. Study objective(.*?)3. Methods', r'Introduction(.*?)Methods', r'Purpose(.*?)Methods', r'Introduction(.*?)Overview', r'Introduction(.*?)Method', r'Background(.*?)Main Text', r'Introduction(.*?)Methodology', r'Introduction(.*?) Related work', r'Context(.*?)Methods', r'Introduction(.*?)Methodology', r'Objectives(.*?)Methods', r'Introduction(.*?)Study Objective', r'Objective(.*?)Method', r'Introduction(.*?)Background' ]), str(doc)).group()           
        print("Introduction is extracted successfully")
        extractedContent.append(intro) 
        return extractedContent 

In [138]:
other_content = []
for i in range(len(finalDocumentSet)):
    content = str(finalDocumentSet[i])
    #Text pre-processing
    content = content.lower()
    content = re.sub(r'\d+', '', content)
    #data = re.sub(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+','', data)
    content = content.replace("\n", "")
    #data = data.strip()
    content = sent_tokenize(content)
    other_content.append(content)

In [139]:
len(other_content)

28

In [140]:
pip install rank-bm25

Note: you may need to restart the kernel to use updated packages.

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [141]:
from rank_bm25 import BM25Okapi

In [142]:
def create_tokens_questions(question):
    tokenized_question = question.split()
    create_chunks = nltk.ne_chunk(nltk.pos_tag(tokenized_question))
    assign_pos_tags = tree2conlltags(create_chunks)
    return assign_pos_tags

In [143]:
def extract_pos(question):
    noun_add = []
    assign_pos_tags = create_tokens_questions(question)
    print(assign_pos_tags)
    for i in range(0, len(assign_pos_tags) - 1):
        if (assign_pos_tags[i][1] == 'NN' and assign_pos_tags[i + 1][1] == 'NN'):
            extract_nouns = assign_pos_tags[i] + assign_pos_tags[i + 1]
            noun_add.append(extract_nouns)
        elif assign_pos_tags[i][1] == 'NNS':
            extract_nouns = assign_pos_tags[i]
            print("The word", extract_nouns[0], "is extracted and added to the query list")
            noun_add.append(extract_nouns[0])
        elif assign_pos_tags[i][1] == 'NN':
            extract_nouns = assign_pos_tags[i]
            print("The word", extract_nouns[0], "is extracted and added to the query list")
            noun_add.append(extract_nouns[0])
    return noun_add

In [144]:
introFinalData = []
global keyword
def extractIntroduction(inputDoc):
    for j in range(len(inputDoc)):
        question = "what is the aim or objective or motivation or focus of this paper?"
        tokenized_corpus = [doc.split(" ") for doc in inputDoc[j]]
        bm25 = BM25Okapi(tokenized_corpus)
        noun_add = extract_pos(question)
        #print(noun_add)
        get_final_results = bm25.get_top_n(noun_add, inputDoc[j], n=10)
        introFinalData.append(get_final_results)
        #print(get_final_results)
    return introFinalData


In [145]:
otherContent = []
global keyword
def extractMethods(inputDoc):
    for j in range(len(inputDoc)):
        question = "what is the research methodology used in this paper?"
        tokenized_corpus = [doc.split(" ") for doc in inputDoc[j]]
        bm25 = BM25Okapi(tokenized_corpus)
        noun_add = extract_pos(question)
        #print(noun_add)
        get_final_results = bm25.get_top_n(noun_add, inputDoc[j], n=10)
        otherContent.append(get_final_results)
        #print(get_final_results)
    return otherContent

In [146]:
Conclu = []
global keyword
def extractConclusion(inputDoc):
    for j in range(len(inputDoc)):
        question = "what is the conclusions and discussions and justifications of this paper?"
        tokenized_corpus = [doc.split(" ") for doc in inputDoc[j]]
        bm25 = BM25Okapi(tokenized_corpus)
        noun_add = extract_pos(question)
        #print(noun_add)
        get_final_results = bm25.get_top_n(noun_add, inputDoc[j], n=10)
        Conclu.append(get_final_results)
        #print(get_final_results)
    return Conclu

In [147]:
#To fetch the conclusion for each paper
QA1 = extractIntroduction(other_content)

[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('or', 'CC', 'O'), ('objective', 'NN', 'O'), ('or', 'CC', 'O'), ('motivation', 'NN', 'O'), ('or', 'CC', 'O'), ('focus', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
The word motivation is extracted and added to the query list
The word focus is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('or', 'CC', 'O'), ('objective', 'NN', 'O'), ('or', 'CC', 'O'), ('motivation', 'NN', 'O'), ('or', 'CC', 'O'), ('focus', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
The word motivation is extracted and added to the query list
The word focus is extracted and added to

The word motivation is extracted and added to the query list
The word focus is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('or', 'CC', 'O'), ('objective', 'NN', 'O'), ('or', 'CC', 'O'), ('motivation', 'NN', 'O'), ('or', 'CC', 'O'), ('focus', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
The word motivation is extracted and added to the query list
The word focus is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('or', 'CC', 'O'), ('objective', 'NN', 'O'), ('or', 'CC', 'O'), ('motivation', 'NN', 'O'), ('or', 'CC', 'O'), ('focus', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to

In [148]:
len(QA1)

28

In [149]:
QA1[1]

['furthermore, it takes the focus off of ac-\\ntual deliverable artifacts and highlights the overall process by which they are developed.',
 'finally,  all  of these code  cards  are  collected into  a  column  of  inte-\\ngrated code to the right.',
 'if  the  receiving \\nplayer meets the condition on the card, they must suffer the consequences described.',
 'these  are  cards  that  are  played  by  one  player  against  the  other.',
 'finally are problem cards, which are at the heart of problems and programmers’ \\ngameplay.',
 'programmer  cards  are  the  player’s  workhorses  and  are  necessary  to  write,  inspect \\nand fix code.',
 'for example, the reusable code concept card allows for a free code \\ncard to be added, while a walkthrough card allows for unclear requirements cards to be re-\\nworked.',
 'concept cards represent decisions that a player may make regarding their \\napproach to the project.',
 'examples  of  each  of  these  are  also \\nshown in figure .',
 'h

In [150]:
#To fetch the methods for each paper
QA2 = extractMethods(other_content)

[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('research', 'NN', 'O'), ('methodology', 'NN', 'O'), ('used', 'VBN', 'O'), ('in', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word methodology is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('research', 'NN', 'O'), ('methodology', 'NN', 'O'), ('used', 'VBN', 'O'), ('in', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word methodology is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('research', 'NN', 'O'), ('methodology', 'NN', 'O'), ('used', 'VBN', 'O'), ('in', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word methodology is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('research', 'NN', 'O'), ('methodology', 'NN', 'O'), ('used', 'VBN', 'O'), ('in', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word metho

In [151]:
QA2

[['ieee comput ;():–.\\n\\n[] toye g, cutkosky mr, leifer l, tenenbaum j, glicksman j. share:\\na methodology and environment for collaborative product develop-\\nment.',
  "in: proceedings of the eighth international conference\\non computer supported collaborative work in design, xiamen,\\nchina, ; p. –.\\n\\n\\xc']",
  ', the agent infrastructures or common\\nfunctionalities are abstracted and associated with each agent\\nrole.',
  ').',
  'the most frequently used protocol\\nis the fipa request protocol (fig.',
  'in\\naade, we only implement a subset of these communicative\\nacts based on the interaction protocols chosen for the related\\nengineering applications.',
  'and to make sure that only the\\nexpected messages are passing to its message queues.\\n\\n.. messaging package\\n\\nfipa deﬁned  communicative acts (performatives) and\\n interaction protocols so that agents can interact under a\\nuniversal message oriented communication language.',
  'simple matching mechanisms ar

In [152]:
QA2[1]

["\\n\\n\\xc']",
 'then they play de-\\nsign cards in a column to the right of this, and then have their programmers create code cards \\nduring implementation.',
 'these  are  cards  that  are  played  by  one  player  against  the  other.',
 'finally are problem cards, which are at the heart of problems and programmers’ \\ngameplay.',
 'programmer  cards  are  the  player’s  workhorses  and  are  necessary  to  write,  inspect \\nand fix code.',
 'for example, the reusable code concept card allows for a free code \\ncard to be added, while a walkthrough card allows for unclear requirements cards to be re-\\nworked.',
 'concept cards represent decisions that a player may make regarding their \\napproach to the project.',
 'examples  of  each  of  these  are  also \\nshown in figure .',
 'here  they  will  find  three \\ntypes  of  cards:  concepts,  programmers  and  problems.',
 'then,  each  player  draws  five  cards  from  the  main  deck.']

In [153]:
#To fetch the conclusion for each paper
QA3 = extractConclusion(other_content)

[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), ('discussions', 'NNS', 'O'), ('and', 'CC', 'O'), ('justifications', 'NNS', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word conclusions is extracted and added to the query list
The word discussions is extracted and added to the query list
The word justifications is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), ('discussions', 'NNS', 'O'), ('and', 'CC', 'O'), ('justifications', 'NNS', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word conclusions is extracted and added to the query list
The word discussions is extracted and added to the query list
The word justifications is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), (

[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), ('discussions', 'NNS', 'O'), ('and', 'CC', 'O'), ('justifications', 'NNS', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word conclusions is extracted and added to the query list
The word discussions is extracted and added to the query list
The word justifications is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), ('discussions', 'NNS', 'O'), ('and', 'CC', 'O'), ('justifications', 'NNS', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word conclusions is extracted and added to the query list
The word discussions is extracted and added to the query list
The word justifications is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusions', 'NNS', 'O'), ('and', 'CC', 'O'), (

In [154]:
QA3[6]

['conclusions and perspectives\\n\\nthis paper ﬁrst introduced six types of process ﬂows within\\nincluding activity ﬂow, product\\nmanufacturing enterprises,\\nﬂow, resource ﬂow, cost ﬂow, cash ﬂow, and proﬁt ﬂow.',
 'comparing with the real data of the\\nairplane design process, the duration of the design process can\\nbe decreased for  h if the design process is scheduled with\\nthe combined rules, hpfs/lrpt/fcfs, and supported by\\nprocess engineering theory with related software tools.\\n\\nafter applying the proposed system in some enterprise\\nprocess evaluation of manufacturing and service enterprises,\\nsome conclusions and recommendations have been drawn as\\nfollowing, more detailed results can be founded in ref.',
 "the simulation of enterprise\\nprocesses will be implemented with the cooperative simulation\\nof multi-agent systems among the sub-process models and the\\nagent-based process simulation within the sub-process model.\\n\\n']",
 'illustrates a schematic chart of

In [155]:
len(QA1)

28

In [156]:
len(QA2)

28

In [157]:
len(QA3)

28

In [158]:
global data_cleaned
def clean(data):
    data_cleaned=[]
    for line in data:
        line = line.lower()
        line = re.sub(r'\d+', '', line)
        #line = re.sub(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+','', line)
        #line = line.translate(str.maketrans('','', string.punctuation))
        #line = line.strip()
        line = line.replace("\n", "")
        line = line.replace("\\n", "")
        #line = sent_tokenize(line)
        
        data_cleaned.append(line)
    return data_cleaned
    

In [159]:
QA1_Data = []
for k in range(len(QA1)):
    CleanedData = (QA1[k])
    CleanedData = clean(CleanedData)
    QA1_Data.append(CleanedData)


In [160]:
%store QA1_Data

Stored 'QA1_Data' (list)


In [161]:
QA2_Data = []
for l in range(len(QA2)):
    CleanedData = (QA2[l])
    CleanedData = clean(CleanedData)
    QA2_Data.append(CleanedData)

In [162]:
%store QA2_Data

Stored 'QA2_Data' (list)


In [163]:
QA3_Data = []
for m in range(len(QA3)):
    CleanedData = (QA3[m])
    CleanedData = clean(CleanedData)
    QA3_Data.append(CleanedData)

In [164]:
%store QA3_Data

Stored 'QA3_Data' (list)


In [165]:
#create a dictionary to save the pdf name and the respective answers for the QA1
#Is the aim and the objective stated clearly?
QA1_Ans = {'PDFName': [], 'QA1_ans': [] }
for c,d in zip(files, QA1_Data):    
    QA1_Ans['PDFName'].append(c)
    QA1_Ans['QA1_ans'].append(d)


In [166]:
#create a dictionary to save the pdf name and the respective answers for the QA2
#This can answer two questions partially 
#Adequate description of the sample used and the methods for identifying and recruiting the sample
#Adequate description of methods used to analyse the data (partially)
QA2_Ans = {'PDFName': [], 'QA2_ans': [] }
for e,f in zip(files, QA2_Data):    
    QA2_Ans['PDFName'].append(e)
    QA2_Ans['QA2_ans'].append(f)

In [167]:
#create a dictionary to save the pdf name and the respective answers for the QA3
#Does the study provide clearly stated findings with credible results and justified conclusions
QA3_Ans = {'PDFName': [], 'QA3_ans': [] }
for g,h in zip(files, QA3_Data):    
    QA3_Ans['PDFName'].append(g)
    QA3_Ans['QA3_ans'].append(h)


In [168]:
#export the answers to the QA1,QA2,QA3 to .xslx

#the final results are exported to the excel to let the user read it and score/assess the paper accordingly
from pandas import DataFrame

df1 = DataFrame(QA1_Ans, columns= ['PDFName', 'QA1_ans'])
export_excel = df1.to_excel (r'E:\MasterThesis\EVALUATION\SLR_evaluation\New folder\SLR1\QA_evaluation\QA1_Answer_userStudy1.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path

df2 = DataFrame(QA2_Ans, columns= ['PDFName', 'QA2_ans'])
export_excel = df2.to_excel (r'E:\MasterThesis\EVALUATION\SLR_evaluation\New folder\SLR1\QA_evaluation\QA2_Answer_userStudy2.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path

df3 = DataFrame(QA3_Ans, columns= ['PDFName', 'QA3_ans'])
export_excel = df3.to_excel (r'E:\MasterThesis\EVALUATION\SLR_evaluation\New folder\SLR1\QA_evaluation\QA3_Answer_userStudy3.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path
